# Quantum State Tomography

Below is a simple code capable of performing quantum state tomography on a random quantum state. First, the quantum state is generated by sending the $|0\rangle$ state through Z, Y, and Z rotations parametrized by random $\theta_1, \theta_2,$ and $ \theta_3$, respectively.  Then, we perform measurements in the Pauli matrices' eigenvector basis so that we can compute the Stokes parameters. Remember that the Stokes parameters are defined as $S_0 = 1$ and $S_i = P_{|\psi_i\rangle} - P_{|\psi_i^\bot\rangle}$ for $i = 1, 2, 3\,$, where $|\psi_i\rangle$ is the eigenvector of eigenvalue 1 of the $i^{th}$ Pauli matrix and $P_{|\psi_i\rangle}$ the probability of measuring said state. We can finally compute the density matrix of the final state as $\rho = \frac{1}{2}\sum_{i = 0}^3 S_i \sigma_i\,$.

In [85]:
import numpy as np
from qibo import Circuit, gates

# Definition of parameters
rng = np.random.default_rng()
theta = rng.uniform(0, np.pi, 3)
S = np.ones(4)
Pauli = np.array([np.eye(2),[[0, 1], [1, 0]],[[0, -1j], [1j, 0]],[[1, 0], [0, -1]]], dtype = np.matrix)

# We first run the circuit without performing any measurements so that we can compare the real density matrix with our approximation
c0 = Circuit(1, density_matrix = True)
c0.add(gates.RZ(0, theta[0]))
c0.add(gates.RY(0, theta[1]))
c0.add(gates.RZ(0, theta[2]))
print("Real density matrix:")
print(c0().state())
print(c0.draw())

# We perform measurements in each of the Pauli matrices eigenvector basis
for i in range(1, 4):
    c = Circuit(1)
    c.add(gates.RZ(0, theta[0]))
    c.add(gates.RY(0, theta[1]))
    c.add(gates.RZ(0, theta[2]))

    # Change of basis
    if(i == 1): c.add(gates.H(0))
    elif(i == 2):
        c.add(gates.S(0).dagger)
        c.add(gates.H(0))

    # Measurement
    c.add(gates.M(0))
    result = c(nshots = 1000000)

    freq = result.frequencies()
    S[i] = (freq['0'] - freq['1'])/(freq['0']+freq['1'])
    print()
    print("Measurement of S_", i, ":",  sep = '')
    print(c.draw())
    print(freq)

# Compute of density matrix
rho = 0.5*np.eye(2)
for i in range(1, 4):
    rho = rho + 0.5*S[i]*Pauli[i]

print()
print("Tomographic approximation of the density matrix")
print(rho)



Real density matrix:
[[0.96239578+0.j         0.07626519-0.17428071j]
 [0.07626519+0.17428071j 0.03760422+0.j        ]]
q0: ─RZ─RY─RZ─

Measurement of S_1:
q0: ─RZ─RY─RZ─H─M─
Counter({'0': 575906, '1': 424094})

Measurement of S_2:
q0: ─RZ─RY─RZ─S─H─M─
Counter({'1': 673505, '0': 326495})

Measurement of S_3:
q0: ─RZ─RY─RZ─M─
Counter({'0': 962833, '1': 37167})

Tomographic approximation of the density matrix
[[0.962833 (0.075906+0.173505j)]
 [(0.075906-0.173505j) 0.037167000000000006]]


For n-qubit states, tomography becomes much more complicated. Instead of 3 Stokes parameters, we need $4^n - 1$. First, a random state is generated by sending the $|0\rangle$ state through a contraption parametrized by random $\theta_i$. Then, we loop over all possible Stokes parameters; to compute $S_{i_1, \dots, i_n}$, we will make projective measurements in the basis $\{|\psi_{i_1, j_1}\rangle\otimes\cdots\otimes|\psi_{i_n, j_n}\rangle\}_{j_1, \dots, j_n}$, where $\{|\psi_{i, j}\rangle\}_j$ are the eigenvectors of the $i^{th}$ Pauli matrix. Remember that $ S_{i_1, \dots, i_n} = \bigotimes_{k = 1}^n \left(P_{|\phi_{i_k}\rangle} \pm P_{|\phi_{i_k}^\bot \rangle }\right) = \sum_{k_1, \dots, k_n} \pm P_{|\phi_{i_{k_1}}\rangle\cdots|\phi_{i_{k_n}}\rangle}\,$. On the tensor product, the + sign is used for 0 indeces and the - sign for 1, 2 or 3 indeces. Thus, on the summatory, the sign will be -1 to the power of the number of $\bot$ terms that don't correspond to a 0 subindex. 

In [3]:
import numpy as np
import qibo
from qibo import Circuit, gates
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, X, Y, Z

#Returns the number such that its binary representation in n bits is the inverse of l's binary representation 
def invert_binary(l, n):
    s = 0
    for i in range(n):
        s += (l%2)*(2**(n - 1 - i))
        l //= 2
    return s

n = 3 #number of qubits
m = 10 #number of rng layers
shots = 1000000 #number of shots
rng = np.random.default_rng()
theta = rng.uniform(0, np.pi, 2*n*m)
rho = 0
c0 = Circuit(n, density_matrix = True)
Pauli = np.array([I, X, Y, Z], dtype = qibo.symbols.Symbol)


#Generates the state and prints its density matrix
for i in range(m): 
    for j in range(n):
        c0.add(gates.RY(j, theta[2*n*i+2*j]))
        c0.add(gates.RZ(j, theta[2*n*i+2*j+1]))
        c0.add(gates.CNOT(j, (j+1)%n))
print("Real density matrix:")
print(c0().state())
print(c0.draw())

#Loop over all possible n-quatrit quaternary codes. 
for k in range(0, 4**n):
    c = Circuit(n)

    #Generates state
    for i in range(m): 
        for j in range(n):
            c.add(gates.RY(j, theta[2*n*i+2*j]))
            c.add(gates.RZ(j, theta[2*n*i+2*j+1]))
            c.add(gates.CNOT(j, (j+1)%n))

    #Change of basis
    for l in range(n):
        if(((k//(4**l))%4) == 1) :
            c.add(gates.H(l))
        elif(((k//(4**l))%4) == 2) :
            c.add(gates.S(l).dagger())
            c.add(gates.H(l))
    
    #Measurement
    c.add(gates.M(*range(n)))
    result = c(nshots = shots)
    freq = result.frequencies(binary = False)

    #Computes the k-th Stokes parameter
    S = 0
    for l in range(2**n):
        sign = 1
        for j in range(n):
            if((l//(2**j))%2 == 1 and (k//(4**j))%4 != 0):
                sign *= -1

        S += sign * freq[invert_binary(l, n)]
    S /= shots

    #Adds the k-th Pauli string term to rho
    aux = k
    P = Pauli[aux%4](0)
    aux //= 4
    for i in range(1, n):
        P = P*Pauli[aux%4](i)
        aux //= 4
    rho = rho + (1/(2**n))*S

#Print results
rho = SymbolicHamiltonian(rho).matrix
print(rho)
print('Fidelity = ', (rho@c0().state()).trace())

Real density matrix:
[[ 7.51925440e-02+6.93889390e-18j -3.28357040e-02-1.01944908e-01j
  -1.52163401e-01-3.59224253e-02j  6.11042352e-02+2.47133755e-02j
  -8.36030970e-03-1.92875672e-02j -1.23475702e-01-8.84679652e-02j
   1.28935023e-03-3.54445498e-03j -4.91483267e-02+5.77456443e-02j]
 [-3.28357040e-02+1.01944908e-01j  1.52554323e-01+3.81639165e-17j
   1.15151055e-01-1.90613923e-01j -6.01895230e-02+7.20521509e-02j
   2.98006397e-02-2.91212586e-03j  1.73863916e-01-1.28773553e-01j
   4.24247400e-03+3.29590344e-03j -5.68281673e-02-9.18514009e-02j]
 [-1.52163401e-01+3.59224253e-02j  1.15151055e-01+1.90613923e-01j
   3.25087038e-01-2.08166817e-17j -1.35460141e-01-2.08193375e-02j
   2.61327686e-02+3.50372403e-02j  2.92136234e-01+1.20039027e-01j
  -9.15868695e-04+7.78870992e-03j  7.18716601e-02-1.40337063e-01j]
 [ 6.11042352e-02-2.47133755e-02j -6.01895230e-02-7.20521509e-02j
  -1.35460141e-01+2.08193375e-02j  5.77780489e-02+7.80625564e-18j
  -1.31331003e-02-1.29260232e-02j -1.29417490e-01-3.

[Qibo 0.2.9|ERROR|2024-07-25 16:28:55]: Symbolic Hamiltonian should be a ``sympy`` expression but is <class 'float'>.


TypeError: Symbolic Hamiltonian should be a ``sympy`` expression but is <class 'float'>.

This code is much faster than the previous one. Instead of measuring in a new basis for each of the $4^n - 1$ Stokes parameters, it only uses $3^n$ settings, corresponding to all the possible Stoke parameters subindeces that don't include 0s. The Stokes parameters whose subindeces include 0s can be computed from the same set of probabilities as those that don't, but share the same non-zero terms. Thus, after each set of probabilities is obtained, the code loops over those subindeces that can be obtained by substituting some of the quatrits by 0. An array that keeps track of the n-quatrit codes already visited helps avoid duplicates. 

In [4]:
import numpy as np
import qibo
from qibo import Circuit, gates
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, X, Y, Z

#Returns the number such that its binary representation in n bits is the inverse of l's binary representation 
def invert_binary(l, n):
    s = 0
    for i in range(n):
        s += (l%2)*(2**(n - 1 - i))
        l //= 2
    return s

#Interpreting n as a code in tertiary and m as a code in binary, returns a code in quaternary
#such that the i-th character is 0 if m's i-th character is, and n's i-th character + 1 otherwise
def and_3_2(n, m):
    if(m == 0): return 0
    return 4*and_3_2(n//3, m//2) + (n%3 + 1)*(m%2)

#Returns the number of ones in the binary representation of n
def number_of_ones(n):
    if(n < 2): return n
    return number_of_ones(n//2) + n%2

n = 2 #number of qubits
m = 1 #number of rng layers
shots = 1000000 #number of shots
rng = np.random.default_rng()
theta = rng.uniform(0, np.pi, 2*n*m)
rho = 0
c0 = Circuit(n, density_matrix = True)
Pauli = np.array([I, X, Y, Z], dtype = qibo.symbols.Symbol)
visited = np.zeros(4**n, dtype = bool)

#Generates the state and prints its density matrix
for i in range(m): 
    for j in range(n):
        c0.add(gates.RY(j, theta[2*n*i+2*j]))
        c0.add(gates.RZ(j, theta[2*n*i+2*j+1]))
        c0.add(gates.CNOT(j, (j+1)%n))
print("Real density matrix:")
print(c0().state())
print(c0.draw())

#Loop over all the posible n-trit tertiary codes.
for k in range(0, 3**n):
    c = Circuit(n)
    
    #Generates state
    for i in range(m): 
        for j in range(n):
            c.add(gates.RY(j, theta[2*n*i+2*j]))
            c.add(gates.RZ(j, theta[2*n*i+2*j+1]))
            c.add(gates.CNOT(j, (j+1)%n))

    #Change of basis
    for l in range(n):
        if(((k//(3**l))%3) == 0) :
            c.add(gates.H(l))
        elif(((k//(3**l))%3) == 1) :
            c.add(gates.S(l).dagger())
            c.add(gates.H(l))
    
    #Measurementt 
    c.add(gates.M(*range(n)))
    result = c(nshots = shots)
    freq = result.frequencies(binary = False)

    #Computes as many of the 4^n - 1 Stokes parameters as possible and sums the corresponding
    #term to rho, but only if the parameter hasn't already been calculated
    for i in range(2**n):
        s = and_3_2(k, i)
        if(not visited[s]):
            visited[s] = True
            S = 0
            for l in range(2**n):
                sign = (-1)**(number_of_ones(i&l)%2)
                S += sign * freq[invert_binary(l, n)]
            S /= shots
            
            P = Pauli[s%4](0)
            s //= 4
            for j in range(1, n):
                P = P*Pauli[s%4](j)
                s //= 4
            rho = rho + (1/(2**n))*S*P


#Print results
rho = SymbolicHamiltonian(rho).matrix
print(rho)
print('Fidelity = ', (rho@c0().state()).trace())

Real density matrix:
[[ 0.27233844-1.08420217e-19j -0.07885349+8.75155075e-02j
  -0.11319419+1.27411414e-01j -0.39400402-2.75908934e-03j]
 [-0.07885349-8.75155075e-02j  0.05095438-2.24971951e-18j
   0.07371795-5.16224201e-04j  0.11319419+1.27411414e-01j]
 [-0.11319419-1.27411414e-01j  0.07371795+5.16224201e-04j
   0.10665624-4.71627945e-18j  0.16247221+1.85478490e-01j]
 [-0.39400402+2.75908934e-03j  0.11319419-1.27411414e-01j
   0.16247221-1.85478490e-01j  0.57005094+0.00000000e+00j]]
q0: ─RY─RZ─o───────X─
q1: ───────X─RY─RZ─o─


[Qibo 0.2.9|WARNING|2024-07-25 16:29:44]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


[[ 0.2723185+0.j        -0.0789855+0.087658j  -0.1128945+0.1274605j
  -0.393752 -0.002808j ]
 [-0.0789855-0.087658j   0.0510425+0.j         0.074118 -0.00137j
   0.1132115+0.1270925j]
 [-0.1128945-0.1274605j  0.074118 +0.00137j    0.1068795+0.j
   0.1621445+0.185244j ]
 [-0.393752 +0.002808j   0.1132115-0.1270925j  0.1621445-0.185244j
   0.5697595+0.j       ]]
Fidelity =  (0.9994378723739814-3.469446951953614e-18j)
